# Train CNN to predict bounding boxes

## Load packages

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import BB_CNN

## Load data

In [2]:
dataset = tf.data.TFRecordDataset('train.record')
#dataset = dataset.shuffle(buffer_size=10000)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

## Train CNN

In [ ]:
num_iter = 1000

feature = {'image/height': tf.FixedLenFeature([], tf.int64),
           'image/width': tf.FixedLenFeature([], tf.int64),
           'image/filename': tf.FixedLenFeature([], tf.string),
           'image/source_id': tf.FixedLenFeature([], tf.string),
           'image/key/sha256': tf.FixedLenFeature([], tf.string),
           'image/encoded': tf.FixedLenFeature([], tf.string),
           'image/format': tf.FixedLenFeature([], tf.string),
           'image/object/bbox/xmin': tf.FixedLenFeature([], tf.float32),
           'image/object/bbox/xmax': tf.FixedLenFeature([], tf.float32),
           'image/object/bbox/ymin': tf.FixedLenFeature([], tf.float32),
           'image/object/bbox/ymax': tf.FixedLenFeature([], tf.float32),
           'image/object/class/text': tf.FixedLenFeature([], tf.string),
           'image/object/class/label': tf.FixedLenFeature([], tf.int64),
           'image/object/difficult': tf.FixedLenFeature([], tf.int64),
           'image/object/truncated': tf.FixedLenFeature([], tf.int64),
           'image/object/view': tf.FixedLenFeature([], tf.string)}
with tf.Session()  as sess:
    bb_net = BB_CNN.BB_CNN(kernel_size = 7 * [3], kernel_stride = 7 * [1], num_filters =  7 * [4],
                           pool_size = 7 * [2], pool_stride = 7 * [2], hidden_dim = [100],
                           dropout = 0.5, weight_scale = 1)
    images = tf.placeholder(tf.float32, [1, 1280, 720, 3])
    train_mode = tf.placeholder(tf.bool)
    target_prob = tf.placeholder(tf.float32, [1])
    target_bb = tf.placeholder(tf.float32, [1, 4])
    bb_net.build(images, train_mode)
    bb_net.predict()
    bb_net.loss(target_prob, target_bb)
    train_step = tf.train.AdamOptimizer(1e-4).minimize(bb_net.loss)
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer)
    for i in range(num_iter):
        ne = sess.run(next_element)
        parsed = tf.parse_single_example(ne, features=feature)
        image = sess.run(tf.image.decode_jpeg(parsed['image/encoded']))
        out = sess.run(parsed)
        xmin = max(out['image/object/bbox/xmin'], 0)
        xmax = min(out['image/object/bbox/xmax'], 1)
        ymin = max(out['image/object/bbox/ymin'], 0)
        ymax = min(out['image/object/bbox/ymax'], 1)
        x = image.reshape((1, 1280, 720, 3)) / 255.
        sess.run(train_step, feed_dict={images: x, train_mode: False, target_prob: [1], target_bb: [[xmin, ymin, xmax - xmin, ymax - ymin]]})
        if (i + 1) % 100 == 0:
            net_loss = np.zeros(10)
            for j in range(10):
                ne = sess.run(next_element)
                parsed = tf.parse_single_example(ne, features=feature)
                image = sess.run(tf.image.decode_jpeg(parsed['image/encoded']))
                out = sess.run(parsed)
                xmin = max(out['image/object/bbox/xmin'], 0)
                xmax = min(out['image/object/bbox/xmax'], 1)
                ymin = max(out['image/object/bbox/ymin'], 0)
                ymax = min(out['image/object/bbox/ymax'], 1)
                x = image.reshape((1, 1280, 720, 3)) / 255.
                net_loss[j] = sess.run(bb_net.loss, feed_dict={images: x, train_mode: False, target_prob: [1], target_bb: [[xmin, ymin, np.log(xmax - xmin), np.log(ymax - ymin)]]})
            print('loss after ' + str(i + 1) + ' iterations = ' + str(np.mean(net_loss)))

loss after 100 iterations = 3.04083926678
loss after 200 iterations = 3.88572058678
loss after 300 iterations = 0.670241892338
loss after 400 iterations = 0.622629034519
loss after 500 iterations = 1.71118837595
loss after 600 iterations = 1.62122370005
